In [ ]:
### The notebook was used on a JAMA project to consolidate multiple "steps" without a verification into a single step
#
# Good references are:
# https://community.jamasoftware.com/communities/community-home/digestviewer/viewthread?MessageKey=1bb49b4b-f5d4-4b49-bfa7-9527e6c45f90&CommunityKey=6c432c09-9859-4cd3-8d2c-ea0a02a8a43d&tab=digestviewer#bm1bb49b4b-f5d4-4b49-bfa7-9527e6c45f90
# https://github.com/JamaSoftware/REST-References/
# https://rapidmicrobiosystems.jamacloud.com/api-docs/
# https://rapidmicrobiosystems.jamacloud.com/api-docs/#/
                

In [1]:
# This example makes use of the excellent library at python-requests.org
import requests
import random
import urllib
import json
import sys

base_url = "https://rapidmicrobiosystems.jamacloud.com/rest/latest/"
#https://rapidmicrobiosystems.jamacloud.com/perspective.req#/testCases/10656?projectId=50
#base_url = "{base url}/rest/latest/"

# Username and password should be stored according
# to your organization's security policies
username = input("JAMA username>")
password = getpass.getpass("JAMA password>")



def get(url):
    response = requests.get(url, auth=(username, password))
    return json.loads(response.text)

# arrgh - the JAMA api only gets up to 50 items - need to do multiple requests
def get_all_data(url):
    # from https://community.jamasoftware.com/blogs/john-lastname/2017/09/29/managing-multiple-pages-of-results-in-the-jama-rest-api

    all_data = []
    allowed_results = 10
    max_results = "maxResults=" + str(allowed_results)
    
    result_count = -1
    start_index = 0
    
    while result_count != 0:
        start_at = "startAt=" + str(start_index)
        
        part_url = url + "?" + start_at + "&" + max_results 
        response = get(part_url)
        
        page_info = response["meta"]["pageInfo"]
        start_index = page_info["startIndex"] + allowed_results
        result_count = page_info["resultCount"]

        all_data.extend(response["data"])
    return all_data

    
    

def put(url, payload):
    response = requests.put(url, json=payload, auth=(username, password))
    return response.status_code

def get_id(string_to_find):
    url = base_url + "abstractitems?contains=" + string_to_find
    json_response = get(url)

    total_results = json_response["meta"]["pageInfo"]["totalResults"]

    if total_results == 1:
        data = json_response["data"]
        item = data[0]
        return item["id"]

    else:
        print("string_to_find wasn't unique")
        sys.exit(1)

def update_item(item_id):
    set_lock_state(True, item_id)
    item = get_item(item_id)
    item["fields"]["description"] += test_results()
    put_item(item_id, item)
    set_lock_state(False, item_id)

def set_lock_state(new_lock_state, item_id):
    payload = {
        "locked": new_lock_state
    }
    url = base_url + "items/" + str(item_id) + "/lock"
    put(url, payload)

def get_item(item_id):
    url = base_url + "items/" + str(item_id)
    json_response = get(url)

    return json_response["data"]

def put_item(item_id, item):
    print("put_item ", item_id)
    url = base_url + "items/" + str(item_id)
    status_code = put(url, item)
    if status_code < 400:
        print("Success", str(status_code))
    else:
        print("Fail", str(status_code))

def test_results():
    template = "<p><strong>Imported test results:</strong></p>Status:&nbsp;"
    test_passed = random.randrange(0,2)
    if test_passed == 1:
        return template + "pass"
    return template + "fail"

#if __name__ == "__main__":
#    main()



JAMA username>chris.winsor


NameError: name 'getpass' is not defined

In [299]:
import json

def combine_test_steps(item_id):
    
    item = get_item(item_id)
    
    newStepList = []
    newStepNum = 0
    subActionNum = 0
    
    for stepNo,stepCo in enumerate(item["fields"]["testCaseSteps"]):
        #print("step " + str(stepNo) + " subAction " + str(subActionNum) + " newStepNum " + str(newStepNum))
        
        if subActionNum == 0:
            newStep = {"action": "<p>", "notes": "", "expectedResult": ""}
            
        newStep["action"] = newStep["action"] + item["fields"]["testCaseSteps"][stepNo]["action"] + "</p>"
        newStep["notes"]  = newStep["notes"]  + item["fields"]["testCaseSteps"][stepNo]["notes"]
        
        #if item["fields"]["testCaseSteps"][stepNo]["expectedResult"] != "n/a":
        if (item["fields"]["testCaseSteps"][stepNo]["expectedResult"] == "n/a") or (item["fields"]["testCaseSteps"][stepNo]["expectedResult"] == '<p>n/a</p>\n') :
            subActionNum = subActionNum + 1
        else:
            newStep["expectedResult"] = item["fields"]["testCaseSteps"][stepNo]["expectedResult"]
            newStepList.append(newStep)
            
            subActionNum = 0
            newStepNum = newStepNum + 1
        
            
    # print the results
    #print("-----------")
    #print(item["fields"]["testCaseSteps"])

    item["fields"]["testCaseSteps"] = newStepList
    #print("-----------")
    #print(item["fields"]["testCaseSteps"])
    
    put_item(item_id, item)
        

In [83]:
# consolidate test steps

#string_to_find = "Duplicate of Duplicate of Duplicate of Audit ReportTest: Method: To have better message when running empty Audit report"
#string_to_find = "14180"
#item_id = get_id(string_to_find)

combine_test_steps(14180)


step 0 subAction 0 newStepNum 0
step 1 subAction 0 newStepNum 1
step 2 subAction 1 newStepNum 1
step 3 subAction 2 newStepNum 1
step 4 subAction 0 newStepNum 2
-----------
[{'action': 'Assume that pre-requisite steps are completed', 'expectedResult': '<p>Yes!</p>\n', 'notes': '<p>blah1</p>\n'}, {'action': '<p>Do not create any thing; method, AA, HR, Samples, Worklist.</p>\n', 'expectedResult': 'n/a', 'notes': '<p>blah2</p>\n'}, {'action': 'Click Audit Report', 'expectedResult': 'n/a', 'notes': '<p>blah3</p>\n'}, {'action': 'Click Methods', 'expectedResult': 'Verify that\nDialog message box displays "No data is available for this audit report\nNo error displays', 'notes': '<p>blah4</p>\n'}, {'action': '<p>A single step action</p>\n', 'expectedResult': '<p>This is the expected result</p>\n', 'notes': '<p>whatever</p>\n'}]

[{'action': '<p>Assume that pre-requisite steps are completed</p>', 'notes': '<p>blah1</p>\n', 'expectedResult': '<p>Yes!</p>\n'}, {'action': '<p><p>Do not create any 

In [92]:
# find the "verifications" item - the tests will be in that
# get one of the tests by name - then look for parent ID
item_id = get_id("SRD1059_CW")
item = get_item(item_id)
print(item)
print("---")
item = get_item(9384)
print(item)

{'id': 10583, 'documentKey': 'GDP-FLD-57', 'globalId': 'GID-48981', 'itemType': 32, 'project': 50, 'createdDate': '2018-02-22T22:18:19.000+0000', 'modifiedDate': '2018-02-22T22:18:19.000+0000', 'lastActivityDate': '2018-02-22T22:18:19.000+0000', 'createdBy': 16, 'modifiedBy': 16, 'fields': {'documentKey': 'GDP-FLD-57', 'globalId': 'GID-48981', 'name': 'SRD1059_CW', 'description': ''}, 'resources': {'self': {'allowed': ['GET', 'PUT', 'PATCH', 'DELETE']}}, 'location': {'sortOrder': 1, 'globalSortOrder': 25367, 'sequence': '1.9.4.2', 'parent': {'item': 9384}}, 'lock': {'locked': False}, 'childItemType': 89, 'type': 'items'}
---
{'id': 9384, 'documentKey': 'GDP-SET-25', 'globalId': 'GID-48396', 'itemType': 31, 'project': 50, 'createdDate': '2018-02-20T19:27:42.000+0000', 'modifiedDate': '2018-02-20T19:27:42.000+0000', 'lastActivityDate': '2018-02-21T17:02:52.000+0000', 'createdBy': 14, 'modifiedBy': 14, 'fields': {'setKey': 'Verfication', 'globalId': 'GID-48396', 'documentKey': 'GDP-SET-25

In [258]:
# login - either a get() or a put() is required to apply username/password
verifications = get(base_url + "items/9384")
print(verifications["data"]["fields"]["name"])


Verifications


In [242]:
# get a list of the tests we need to update

verifications = get(base_url + "items/9384")
print(verifications["data"]["fields"]["name"])

verif_children = get_all_data(base_url + "items/9384/children")

for vidx,vc in enumerate(verif_children):
    print(str(vidx), " ", vc["id"], " ", vc["fields"]["name"])
    
    srd_children = get_all_data(base_url + "items/" +  str(vc["id"]) + "/children")
    for sidx,sc in enumerate(srd_children):
        print("~~~~~~", sc["id"], " ", sc["fields"]["name"])





Verifications
0   9514   SRD 1029 TFS 653 - Reports - Need better error message for 'no data' when running empty Audit Report is fixed
~~~~~~ 9550   Overview
~~~~~~ 9551   Audit Report - Method
~~~~~~ 9552   Audit Report - Action Alert Level
~~~~~~ 9553   Audit Report - Handling Rule
~~~~~~ 9554   Audit Report - Samples
~~~~~~ 9555   Audit Report - Worklist
~~~~~~ 9556   Audit Report - General / IT Settings
~~~~~~ 9557   Audit Report - Incubator Settings
~~~~~~ 9558   Audit Report - Users
~~~~~~ 9559   Audit Report - User Roles
~~~~~~ 9560   Audit Report - User Activity
1   10583   SRD1059_CW
~~~~~~ 10584   Overview
~~~~~~ 10585   Test: Help about dialog box
2   10604   SRD1002_CW
~~~~~~ 10605   Overview
~~~~~~ 10606   Test: Test Results by Loaded Date - Title and footer are matching
3   10654   SRD1029_CW
~~~~~~ 10655   Overview
~~~~~~ 10656   Audit ReportTest: Method: To have better message when running empty Audit report
~~~~~~ 10657   Audit ReportTest: Action Alert Level: To have b

~~~~~~ 10947   Overview
~~~~~~ 10948   Approve PageTest: Test Comment Field vertical scroll bar display
~~~~~~ 10949   Approve PageTest: Pass/ Cancelled Comment Field vertical bar display
~~~~~~ 10950   Approve PageTest: General Field vertical scroll bar display
~~~~~~ 10951   Approve PageTest: Test Comment Field vertical scroll bar display
~~~~~~ 10952   Approve PageTest: Pass/ Cancelled Comment Field vertical bar display
~~~~~~ 10953   Approve PageTest: General Field vertical scroll bar display
~~~~~~ 10954   Approve PageTest: Test Comment Field vertical scroll bar display
~~~~~~ 10955   Approve PageTest: Pass/ Cancelled Comment Field vertical bar display
~~~~~~ 10956   Approve PageTest: General Field vertical scroll bar display
~~~~~~ 10957   Approve PageTest: Test Comment Field vertical scroll bar display
~~~~~~ 10958   Approve PageTest: Pass/ Cancelled Comment Field vertical bar display
~~~~~~ 10959   Approve PageTest: General Field vertical scroll bar display
6   10961   SRD1039_

~~~~~~ 11550   Overview - SRD1011 System shall support "Sample ID" barcode label with unlimited length SampleID and Batch/LOT
~~~~~~ 11553   Test 0001 Short Sample ID length and short Lot/Batch
~~~~~~ 13666   Test 0006 Short Sample ID length and short Lot/Batch
~~~~~~ 13704   Test 0011 Short Sample ID length and short Lot/Batch
~~~~~~ 13646   Test 0002 Long Sample ID length -32 and short Lot/Batch
~~~~~~ 13676   Test 0007 Long Sample ID length -32 and short Lot/Batch
~~~~~~ 13777   Test 0012 Long Sample ID length -32 and short Lot/Batch
~~~~~~ 11554   Test 0003 Long Sample ID length(32) and Long Lot/Batch (18)
~~~~~~ 13678   Test 0008 Long Sample ID length(32) and Long Lot/Batch (18)
~~~~~~ 11555   Test 0004 Long Sample ID length(50) and Long Lot/Batch (25)
~~~~~~ 13680   Test 0009 Long Sample ID length(50) and Long Lot/Batch (25)
~~~~~~ 11556   Test 0005 Short Sample ID length(10) and Long Lot/Batch (25)
~~~~~~ 13682   Test 0010 Short Sample ID length(10) and Long Lot/Batch (25)
~~~~~

~~~~~~ 11713   Overview
~~~~~~ 11714   LIMS Simulator: How to create LIMS request
~~~~~~ 11715   LIMS Instrument: Test: Loading LIMS cassettes on none LIMS system
~~~~~~ 11716   LIMS Instrument: Test: Loading LIMS cassettes with no order tests
43   11718   SRD1073
~~~~~~ 11719   Overview
~~~~~~ 11720   Test: Unlock left carousel door while it is closed
~~~~~~ 11721   Test: Unlock left carousel door while it is open
~~~~~~ 11722   Test: Unlock left carousel door while it is open/close
~~~~~~ 11723   Test: Unlock right carousel door while it is closed
~~~~~~ 11724   Test: Unlock right carousel door while it is open
~~~~~~ 11725   Test: Unlock right carousel door while it is open/close
44   11727   SRD1071 Custom Test Report doesn't display redundant Test Data
~~~~~~ 11729   Overview 0001 - Custom Report Test Overview
~~~~~~ 11730   Test 0002 Active Database - One Day Range selection  and cassette has no sample ID or LIMS
~~~~~~ 13584   Test 0038 Active Database - One Day Range selection 

~~~~~~ 9142   Test Help about button
~~~~~~ 9145   Verify information in the About Box
48   13071   SRD1009 Change log/UI Text "dummy" to "temporary", Normally used as "dummy cassette" or "dummy record"
~~~~~~ 13072   Test Text "dummy" is replaced with "temporary" in GDSystem.log
49   9051   Archive
~~~~~~ 8948   Multiple Exposure Feature_SRD1040
50   14184   TEST001
~~~~~~ 14185   This is test verification 1 single step
~~~~~~ 14186   This is a test verification 2
~~~~~~ 14187   This is a test verification 3


In [236]:
import json
import re

def remove_p_from_item_name(item_id):
    print(item_id)

    item = get_item(item_id)
    #print(item)
    
    temp = item["fields"]["name"]
    #print("before|", temp, "|")
    temp = re.sub('\n','',temp)
    #print("after |", temp, "|")

    item["fields"]["name"] = temp + str('xxxxx')
    put_item(item_id, item)
    item["fields"]["name"] = temp
    put_item(item_id, item)
        

In [206]:
# clean up the test names (remove /n)

#remove_p_from_item_name(10656)



#remove_p_from_item_name(10657)

remove_p_from_item_name(10658)
remove_p_from_item_name(10659)
remove_p_from_item_name(10660)
remove_p_from_item_name(10661)
remove_p_from_item_name(10662)
remove_p_from_item_name(10663)
remove_p_from_item_name(10664)
remove_p_from_item_name(10665)
remove_p_from_item_name(10859)
remove_p_from_item_name(10864)
remove_p_from_item_name(10869)
remove_p_from_item_name(10874)
remove_p_from_item_name(10879)
remove_p_from_item_name(10884)
remove_p_from_item_name(10889)
remove_p_from_item_name(10894)
remove_p_from_item_name(10899)
remove_p_from_item_name(10904)
remove_p_from_item_name(10909)
remove_p_from_item_name(10914)
remove_p_from_item_name(10919)
remove_p_from_item_name(10924)
remove_p_from_item_name(10929)
remove_p_from_item_name(10934)
remove_p_from_item_name(10939)
remove_p_from_item_name(10944)
remove_p_from_item_name(10948)
remove_p_from_item_name(10949)
remove_p_from_item_name(10950)
remove_p_from_item_name(10951)
remove_p_from_item_name(10952)
remove_p_from_item_name(10953)
remove_p_from_item_name(10954)
remove_p_from_item_name(10955)
remove_p_from_item_name(10956)
remove_p_from_item_name(10957)
remove_p_from_item_name(10958)
remove_p_from_item_name(10959)
remove_p_from_item_name(10967)
remove_p_from_item_name(10971)
remove_p_from_item_name(10972)
remove_p_from_item_name(10973)
remove_p_from_item_name(10974)
remove_p_from_item_name(10975)
remove_p_from_item_name(10976)
remove_p_from_item_name(10977)
remove_p_from_item_name(10978)
remove_p_from_item_name(10979)
remove_p_from_item_name(10983)
remove_p_from_item_name(10984)
remove_p_from_item_name(10985)
remove_p_from_item_name(10989)
remove_p_from_item_name(10990)
remove_p_from_item_name(10994)
remove_p_from_item_name(10995)
remove_p_from_item_name(10996)
remove_p_from_item_name(10997)
remove_p_from_item_name(10998)
remove_p_from_item_name(10999)
remove_p_from_item_name(11003)
remove_p_from_item_name(11004)
remove_p_from_item_name(11005)
remove_p_from_item_name(11006)
remove_p_from_item_name(11012)
remove_p_from_item_name(11013)
remove_p_from_item_name(11014)
remove_p_from_item_name(11015)
remove_p_from_item_name(11016)
remove_p_from_item_name(11017)
remove_p_from_item_name(11018)
remove_p_from_item_name(11019)
remove_p_from_item_name(11023)
remove_p_from_item_name(11029)
remove_p_from_item_name(11030)
remove_p_from_item_name(11031)
remove_p_from_item_name(11032)
remove_p_from_item_name(11033)
remove_p_from_item_name(11083)
remove_p_from_item_name(11084)
remove_p_from_item_name(11541)
remove_p_from_item_name(11565)
remove_p_from_item_name(11566)
remove_p_from_item_name(11567)
remove_p_from_item_name(11568)
remove_p_from_item_name(11569)
remove_p_from_item_name(11571)
remove_p_from_item_name(11572)
remove_p_from_item_name(11573)
remove_p_from_item_name(11574)
remove_p_from_item_name(11575)
remove_p_from_item_name(11583)
remove_p_from_item_name(11584)
remove_p_from_item_name(11585)
remove_p_from_item_name(11586)
remove_p_from_item_name(11587)
remove_p_from_item_name(11588)
remove_p_from_item_name(11589)
remove_p_from_item_name(11590)
remove_p_from_item_name(11591)
remove_p_from_item_name(11605)
remove_p_from_item_name(11606)
remove_p_from_item_name(11607)
remove_p_from_item_name(11608)
remove_p_from_item_name(11612)
remove_p_from_item_name(11613)
remove_p_from_item_name(11617)
remove_p_from_item_name(11618)
remove_p_from_item_name(11619)
remove_p_from_item_name(11623)
remove_p_from_item_name(11627)
remove_p_from_item_name(11628)
remove_p_from_item_name(11629)
remove_p_from_item_name(11630)
remove_p_from_item_name(11631)
remove_p_from_item_name(11635)
remove_p_from_item_name(11645)
remove_p_from_item_name(11646)
remove_p_from_item_name(11647)
remove_p_from_item_name(11648)
remove_p_from_item_name(11649)
remove_p_from_item_name(11650)
remove_p_from_item_name(11652)
remove_p_from_item_name(11653)



10658
Success
10659
Success
10660
Success
10661
Success
10662
Success
10663
Success
10664
Success
10665
Success
10859
Success
10864
Success
10869
Success
10874
Success
10879
Success
10884
Success
10889
Success
10894
Success
10899
Success
10904
Success
10909
Success
10914
Success
10919
Success
10924
Success
10929
Success
10934
Success
10939
Success
10944
Success
10948
Success
10949
Success
10950
Success
10951
Success
10952
Success
10953
Success
10954
Success
10955
Success
10956
Success
10957
Success
10958
Success
10959
Success
10967
Success
10971
Success
10972
Success
10973
Success
10974
Success
10975
Success
10976
Success
10977
Success
10978
Success
10979
Success
10983
Success
10984
Success
10985
Success
10989
Success
10990
Success
10994
Success
10995
Success
10996
Success
10997
Success
10998
Success
10999
Success
11003
Success
11004
Success
11005
Success
11006
Success
11012
Success
11013
Success
11014
Success
11015
Success
11016
Success
11017
Success
11018
Success
11019
Success
11023


In [243]:
#    print(get_item(item_id))

#print(get_item(11023)['fields']['name'])
#print(get_item(11023))
#remove_p_from_item_name(11023)
#print(get_item(11023)['fields']['name'])
#print(get_item(11023))

#print(get_item(11538)['fields']['name'])
#print(get_item(11538))
#remove_p_from_item_name(11538)
#print(get_item(11538)['fields']['name'])
#print(get_item(11538))

#print(get_item(11565)['fields']['name'])
#print(get_item(11565))
#remove_p_from_item_name(11565)
#print(get_item(11565)['fields']['name'])
#print(get_item(11565))

#remove_p_from_item_name(11566)
#remove_p_from_item_name(11567)
#remove_p_from_item_name(11568)
#remove_p_from_item_name(11569)
#remove_p_from_item_name(11571)
#remove_p_from_item_name(11572)
#remove_p_from_item_name(11573)
#remove_p_from_item_name(11574)
#remove_p_from_item_name(11575)

#remove_p_from_item_name(10944)
#remove_p_from_item_name(11547)
#remove_p_from_item_name(11707)
#remove_p_from_item_name(13737)

remove_p_from_item_name(11651)


11651
Success 200
Success 200


In [324]:
# consolidate test steps

# login - either a get() or a put() is required to apply username/password
verifications = get(base_url + "items/9384")
print(verifications["data"]["fields"]["name"])


#0   9514   SRD 1029 TFS 653 - Reports - Need better error message for 'no data' when running empty Audit Report is fixed

#set_lock_state(False, 9551)
#combine_test_steps(9551)
#combine_test_steps(9552  )
#combine_test_steps(9553  )
#combine_test_steps(9554  )
#combine_test_steps(9555  )
#combine_test_steps(9556  )
#combine_test_steps(9557  )
#combine_test_steps(9558  )
#combine_test_steps(9559  )
#combine_test_steps(9560  )

#1   10583   SRD1059_CW
#combine_test_steps(10585  )

#2   10604   SRD1002_CW
#combine_test_steps(10606  )

#3   10654   SRD1029_CW
#combine_test_steps(10656  )
#combine_test_steps(10657  )
#combine_test_steps(10658  )
#combine_test_steps(10659  )
#combine_test_steps(10660  )
#combine_test_steps(10661  )
#combine_test_steps(10662  )
#combine_test_steps(10663  )
#combine_test_steps(10664  )
#combine_test_steps(10665  )

#4   10853   SRD1032_CW
#combine_test_steps(10855  )
#combine_test_steps(10856  )
#combine_test_steps(10857  )
#combine_test_steps(10858  )
#combine_test_steps(10859  )
#combine_test_steps(10860  )
#combine_test_steps(10861  )
#combine_test_steps(10862  )
#combine_test_steps(10863  )
#combine_test_steps(10864  )
#combine_test_steps(10865  )
#combine_test_steps(10866  )
#combine_test_steps(10867  )
#combine_test_steps(10868  )
#combine_test_steps(10869  )
#combine_test_steps(10870  )
#combine_test_steps(10871  )
#combine_test_steps(10872  )
#combine_test_steps(10873  )
#combine_test_steps(10874  )
#combine_test_steps(10875  )
#combine_test_steps(10876  )
#combine_test_steps(10877  )
#combine_test_steps(10878  )
#combine_test_steps(10879  )
#combine_test_steps(10880  )
#combine_test_steps(10881  )
#combine_test_steps(10882  )
#combine_test_steps(10883  )
#combine_test_steps(10884  )
#combine_test_steps(10885  )
#combine_test_steps(10886  )
#combine_test_steps(10887  )
#combine_test_steps(10888  )
#combine_test_steps(10889  )
#combine_test_steps(10890  )
#combine_test_steps(10891  )
#combine_test_steps(10892  )
#combine_test_steps(10893  )
#combine_test_steps(10894  )
#combine_test_steps(10895  )
#combine_test_steps(10896  )
#combine_test_steps(10897  )
#combine_test_steps(10898  )
#combine_test_steps(10899  )
#combine_test_steps(10900  )
#combine_test_steps(10901  )
#combine_test_steps(10902  )
#combine_test_steps(10903  )
#combine_test_steps(10904  )
#combine_test_steps(10905  )
#combine_test_steps(10906  )
#combine_test_steps(10907  )
#combine_test_steps(10908  )
#combine_test_steps(10909  )
#combine_test_steps(10910  )
#combine_test_steps(10911  )
#combine_test_steps(10912  )
#combine_test_steps(10913  )
#combine_test_steps(10914  )
#combine_test_steps(10915  )
#combine_test_steps(10916  )
#combine_test_steps(10917  )
#combine_test_steps(10918  )
#combine_test_steps(10919  )
#combine_test_steps(10920  )
#combine_test_steps(10921  )
#combine_test_steps(10922  )
#combine_test_steps(10923  )
#combine_test_steps(10924  )
#combine_test_steps(10925  )
#combine_test_steps(10926  )
#combine_test_steps(10927  )
#combine_test_steps(10928  )
#combine_test_steps(10929  )
#combine_test_steps(10930  )
#combine_test_steps(10931  )
#combine_test_steps(10932  )
#combine_test_steps(10933  )
#combine_test_steps(10934  )
#combine_test_steps(10935  )
#combine_test_steps(10936  )
#combine_test_steps(10937  )
#combine_test_steps(10938  )
#combine_test_steps(10939  )
#combine_test_steps(10940  )
#combine_test_steps(10941  )
#combine_test_steps(10942  )
#combine_test_steps(10943  )
#combine_test_steps(10944  )

#5   10946   SRD1036_CW
#combine_test_steps(10948  )
#combine_test_steps(10949  )
#combine_test_steps(10950  )
#combine_test_steps(10951  )
#combine_test_steps(10952  )
#combine_test_steps(10953  )
#combine_test_steps(10954  )
#combine_test_steps(10955  )
#combine_test_steps(10956  )
#combine_test_steps(10957  )
#combine_test_steps(10958  )
#combine_test_steps(10959  )

#6   10961   SRD1039_CW
#combine_test_steps(10963  )

#7   10965   SRD1037_CW
#combine_test_steps(10967  )

#8   10969   SRD1041_CW
#combine_test_steps(10971  )
#combine_test_steps(10972  )
#combine_test_steps(10973  )
#combine_test_steps(10974  )
#combine_test_steps(10975  )
#combine_test_steps(10976  )
#combine_test_steps(10977  )
#combine_test_steps(10978  )
#combine_test_steps(10979  )

#9   10981   SRD1043_CW
#combine_test_steps(10983  )
#combine_test_steps(10984  )
#combine_test_steps(10985  )

#10   10987   SRD1044_CW
#combine_test_steps(10989  )
#combine_test_steps(10990  )

#11   10992   SRD1046_CW
#combine_test_steps(10994  )
#combine_test_steps(10995  )
#combine_test_steps(10996  )
#combine_test_steps(10997  )
#combine_test_steps(10998  )
#combine_test_steps(10999  )

#12   11830   SRD1049_CW
#combine_test_steps(11832  )

#13   11001   SRD1050_CW
#combine_test_steps(11003  )
#combine_test_steps(11004  )
#combine_test_steps(11005  )
#combine_test_steps(11006  )

#14   11008   SRD1058_CW
#combine_test_steps(11010  )
#combine_test_steps(11011  )
#combine_test_steps(11012  )
#combine_test_steps(11013  )
#combine_test_steps(11014  )
#combine_test_steps(11015  )
#combine_test_steps(11016  )
#combine_test_steps(11017  )
#combine_test_steps(11018  )
#combine_test_steps(11019  )

#15   11021   SRD1006_CW
#combine_test_steps(11023  )
#combine_test_steps(11024  )
#combine_test_steps(11025  )
#combine_test_steps(11026  )
#combine_test_steps(11027  )
#combine_test_steps(11028  )
#combine_test_steps(11029  )
#combine_test_steps(11030  )
#combine_test_steps(11031  )
#combine_test_steps(11032  )
#combine_test_steps(11033  )

#16   11035   SRD1005_CW
#combine_test_steps(11037  )

#17   11039   SRD1028_CW
#combine_test_steps(11041  )

#18   11043   SRD1001_CW
#combine_test_steps(11045  )
#combine_test_steps(11046  )
#combine_test_steps(11047  )

#19   11081   SRD1060_CW
#combine_test_steps(11083  )
#combine_test_steps(11084  )
#combine_test_steps(11085  )
#combine_test_steps(11086  )
#combine_test_steps(11087  )
#combine_test_steps(11088  )

#20   11090   SRD1026_CW
#combine_test_steps(11092  )

#21   11118   SRD1076_CW
#combine_test_steps(11120  )
#combine_test_steps(11121  )
#combine_test_steps(11122  )
#combine_test_steps(11123  )
#combine_test_steps(11124  )
#combine_test_steps(11125  )
#combine_test_steps(11126  )
#combine_test_steps(11127  )
#combine_test_steps(11128  )
#combine_test_steps(11129  )
#combine_test_steps(11130  )

#22   11132   SRD1068
#combine_test_steps(11134  )
#combine_test_steps(11135  )
#combine_test_steps(11136  )
#combine_test_steps(11137  )
#combine_test_steps(11138  )

#23   11193   SRD1000
#combine_test_steps(11195  )
#combine_test_steps(11196  )
#combine_test_steps(12330  )
#combine_test_steps(11198  )

#24   11535   SRD1004
#combine_test_steps(11537  )
#combine_test_steps(11538  )
#combine_test_steps(11539  )
#combine_test_steps(11540  )
#combine_test_steps(11541  )
#combine_test_steps(11542  )
#combine_test_steps(11543  )
#combine_test_steps(11544  )
#combine_test_steps(11545  )
#combine_test_steps(11546  )
#combine_test_steps(11547  )

#26   11577   SRD1030
#combine_test_steps(11579  )

#27   11592   SRD1033
#combine_test_steps(11594  )
#combine_test_steps(11595  )
#combine_test_steps(11596  )
#combine_test_steps(11597  )

#28   11599   SRD1035
#combine_test_steps(11601  )

#29   11581   SRD1031
#combine_test_steps(11583  )
#combine_test_steps(11584  )
#combine_test_steps(11585  )
#combine_test_steps(11586  )
#combine_test_steps(11587  )
#combine_test_steps(11588  )
#combine_test_steps(11589  )
#combine_test_steps(11590  )
#combine_test_steps(11591  )

#30   11603   SRD1038
#combine_test_steps(11605  )
#combine_test_steps(11606  )
#combine_test_steps(11607  )
#combine_test_steps(11608  )

#31   11610   SRD1042
#combine_test_steps(11612  )
#combine_test_steps(11613  )

#32   11615   SRD1045
#combine_test_steps(11617  )
#combine_test_steps(11618  )
#combine_test_steps(11619  )

#33   11621   SRD1047
#combine_test_steps(11623  )

#34   11625   SRD1056
#combine_test_steps(11627  )
#combine_test_steps(11628  )
#combine_test_steps(11629  )
#combine_test_steps(11630  )
#combine_test_steps(11631  )

#35   11633   SRD1051_SRD1057
#combine_test_steps(11635  )

#36   11637   SRD1061
#combine_test_steps(11639  )
#combine_test_steps(11640  )
#combine_test_steps(11641  )

#37   11643   SRD1072
#combine_test_steps(11645  )
#combine_test_steps(11646  )
#combine_test_steps(11647  )
#combine_test_steps(11648  )
#combine_test_steps(11649  )
#combine_test_steps(11650  )
#combine_test_steps(11651  )
#combine_test_steps(11652  )
#combine_test_steps(11653  )

#38   11655   SRD1028_CW
#combine_test_steps(11657  )

#39   11661   SRD1109
#combine_test_steps(11664  )
#combine_test_steps(11665  )
#combine_test_steps(11666  )
#combine_test_steps(11667  )
#combine_test_steps(11668  )
#combine_test_steps(11669  )
#combine_test_steps(11670  )
#combine_test_steps(11671  )
#combine_test_steps(11672  )
#combine_test_steps(11673  )
#combine_test_steps(11674  )
#combine_test_steps(11675  )
#combine_test_steps(11676  )
#combine_test_steps(11677  )
#combine_test_steps(11678  )
#combine_test_steps(11679  )
#combine_test_steps(11680  )
#combine_test_steps(11681  )
#combine_test_steps(11682  )
#combine_test_steps(11683  )
#combine_test_steps(11684  )
#combine_test_steps(11685  )
#combine_test_steps(11686  )

#40   11694   SRD1083
#combine_test_steps(11696  )
#combine_test_steps(11697  )
#combine_test_steps(11698  )
#combine_test_steps(11699  )
#combine_test_steps(11700  )

#41   11702   SRD1048
#combine_test_steps(11704  )
#combine_test_steps(11705  )
#combine_test_steps(11706  )
#combine_test_steps(11707  )

#42   11709   SRD1010
#combine_test_steps(11714  )
#combine_test_steps(11715  )
#combine_test_steps(11716  )

#43   11718   SRD1073
#combine_test_steps(11720  )
#combine_test_steps(11721  )
#combine_test_steps(11722  )
#combine_test_steps(11723  )
#combine_test_steps(11724  )
#combine_test_steps(11725  )

#45   11846   SRD1056
#combine_test_steps(11848  )
#combine_test_steps(11849  )
#combine_test_steps(11850  )





Verifications
put_item  11848
Success 200
put_item  11849
Success 200
put_item  11850
Success 200
